# Two federal programs at odds
## School lunch debt in Durham Public Schools and competition between CEP and Title I

In [164]:
# Load packages
require(ggplot2)
require(dplyr)
require(tidyr)
require(knitr)
require(stringr)

# Load lunch debt and 2018-19 banding data
debt = read.csv('./data/meal_debt.csv')
bands = read.csv('./data/banding_1718.csv')

# Load CEP eligibility data
# File names as list
cep.list = list.files('./data') %>%
    grep('cep', ., value=TRUE) %>%
    paste('./data/', ., sep='')

# Read files
cep.dat = lapply(cep.list, function(x) read.csv(x))

# Add year
year = c('2014-15', '2015-16', '2016-17', '2017-18', '2018-19')
for(i in 1:length(cep.dat)) {
  cep.dat[[i]]$year = year[i]
}
                 
# Merge to one dataset
cep = do.call(plyr::rbind.fill, cep.dat)
                 
# Filter to Durham
cep = cep %>%
    filter(lea_no == 320)

# Add missing school numbers for 2014-15 CEP eligibility data
school_no_key = cep %>%
  filter(!is.na(school_no)) %>%
  select(school_name, school_no) %>%
  unique()
school_key = school_no_key$school_no
names(school_key) = school_no_key$school_name  

cep$school_no[cep$year=='2014-15'] = school_key[cep$school_name[cep$year=='2014-15']] %>% unlist() %>% as.vector()           

# Control for data types
cep$school_no = cep$school_no %>% as.character()
debt$school_no = debt$school_no %>% as.character()
                 
# Indicator for CEP eligibility, participation
cep$eligible[grep('X', cep$eligible)] = 1
cep$eligible[cep$eligible!=1] = 0
cep$eligible = cep$eligible %>% as.numeric()

cep$participating[grep('X', cep$participating)] = 1
cep$participating[cep$participating!=1] = 0
cep$participating = cep$participating %>% as.numeric()

# Remove unnecessary columns
cep = cep %>% subset(., select=-c(lea_name, lea_no))

# Control for conflicting school names                 
debt = debt %>% subset(., select=-c(school_name))
                 
# Merge lunch debt and CEP data
df = left_join(debt, cep, by = c('school_no', 'year'))
                 
# Fill missing names from CEP data
school_key2 = names(school_key)
names(school_key2) = school_no_key$school_no
df$school_name[is.na(df$school_name)] = school_key2[df$school_no[is.na(df$school_name)]] %>% unlist() %>% as.vector()
                 
# remove DPS hospital school since non-standard school
df = df[-which(df$school_name=="DPS Hospital School"),]

# remove 389, 700 since no longer active
# remove 353, housed in Durham Tech
df = df[-which(df$school_no %in% c(389, 700, 353)),]

### School lunch debt and CEP

In [165]:
df %>%
    group_by(year) %>%
    summarise(debt = sum(unpaid))

year,debt
<chr>,<dbl>
2010-11,204692.22
2011-12,112706.47
2012-13,117526.02
2013-14,108231.05
2014-15,85092.75
2015-16,78428.06
2016-17,127939.71
2017-18,209021.50


Last school year, Durham public schools had just over $209,000 in school lunch debt--the most the district has seen in the past eight years.

In [166]:
df %>%
    group_by(year) %>%
    summarise(eligible = sum(eligible, na.rm = TRUE),
             paricipating = sum(participating, na.rm = TRUE))

year,eligible,paricipating
<chr>,<dbl>,<dbl>
2010-11,0,0
2011-12,0,0
2012-13,0,0
2013-14,0,0
2014-15,25,0
2015-16,27,10
2016-17,28,12
2017-18,25,13


Last year, 12 of the 25 schools eligible for CEP did not participate in the program.

In [167]:
# What percent of schools are eligible and not participating in CEP?
n = df %>%
    filter(year=='2017-18') %>%
    nrow()
12/n

# Percent of debt
enp_debt = df %>%
    filter(year=='2017-18',
          eligible==1,
          participating==0) %>%
    select(unpaid) %>%
    sum(.)
debt_1718 = sum(df$unpaid[df$year=='2017-18'])
enp_debt/debt_1718

[1] 0.2352941

[1] 0.3811623

Last year, 24 percent of schools account for just under 40 percent of all unpaid meals. They were all eligible, but not participating in CEP.

### CEP and Title I